<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/Multivariate-Time-Series-Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

# Loading the dataset

In [2]:
tf.keras.utils.get_file(
    "ridership.tgz",
    "https://github.com/TirendazAcademy/Deep-Learning-with-TensorFlow/raw/main/Data/ridership.tgz",
    cache_dir=".",
    extract=True
)

108512/108512 [==============================] - 0s 0us/step


'./datasets/ridership.tgz'

# Data preprocessing

In [3]:
import pandas as pd
from pathlib import Path

path = Path("datasets/ridership/CTA_-_Ridership_-_Daily_Boarding_Totals.csv")
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)
df = df.drop_duplicates()

In [4]:
df.head()

,day_type,bus,rail
date,,,
2001-01-01,U,297192,126455
2001-01-02,W,780827,501952
2001-01-03,W,824923,536432
2001-01-04,W,870021,550011
2001-01-05,W,890426,557917


In [5]:
df.dtypes

day_type    object
bus          int64
rail         int64
dtype: object

In [6]:
df_mulvar = df[["bus", "rail"]] / 1e6
df_mulvar["next_day_type"] = df["day_type"].shift(-1)
df_mulvar = pd.get_dummies(df_mulvar)

In [7]:
df_mulvar.head()

,bus,rail,next_day_type_A,next_day_type_U,next_day_type_W
date,,,,,
2001-01-01,0.297192,0.126455,0,0,1
2001-01-02,0.780827,0.501952,0,0,1
2001-01-03,0.824923,0.536432,0,0,1
2001-01-04,0.870021,0.550011,0,0,1
2001-01-05,0.890426,0.557917,1,0,0


In [8]:
mulvar_train = df_mulvar["2016-01":"2018-12"]
mulvar_valid = df_mulvar["2019-01":"2019-05"]
mulvar_test = df_mulvar["2019-06":]

In [9]:
tf.random.set_seed(42)

seq_length =  56
train_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.to_numpy(),
    targets=mulvar_train["rail"][seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.to_numpy(),
    targets=mulvar_valid["rail"][seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

# Simple RNN

In [10]:
tf.random.set_seed(42)
mulvar_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    tf.keras.layers.Dense(1)
])

In [11]:
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=500):
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=50, restore_best_weights=True)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    history = model.fit(train_set, validation_data=valid_set, epochs=epochs,
                        callbacks=[early_stopping_cb])
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae * 1e6

In [12]:
fit_and_evaluate(mulvar_model, train_mulvar_ds, valid_mulvar_ds,
                 learning_rate=0.05)

Epoch 1/500
33/33 [==============================] - 3s 52ms/step - loss: 0.0336 - mae: 0.1906 - val_loss: 0.0017 - val_mae: 0.0457
Epoch 2/500
33/33 [==============================] - 1s 31ms/step - loss: 0.0031 - mae: 0.0616 - val_loss: 0.0013 - val_mae: 0.0364
Epoch 3/500
33/33 [==============================] - 1s 29ms/step - loss: 0.0018 - mae: 0.0425 - val_loss: 9.8101e-04 - val_mae: 0.0324
Epoch 4/500
33/33 [==============================] - 1s 28ms/step - loss: 0.0017 - mae: 0.0413 - val_loss: 0.0013 - val_mae: 0.0408
Epoch 5/500
33/33 [==============================] - 1s 29ms/step - loss: 0.0016 - mae: 0.0393 - val_loss: 9.3749e-04 - val_mae: 0.0323
Epoch 6/500
33/33 [==============================] - 1s 26ms/step - loss: 0.0015 - mae: 0.0389 - val_loss: 0.0012 - val_mae: 0.0378
Epoch 7/500
33/33 [==============================] - 1s 33ms/step - loss: 0.0015 - mae: 0.0385 - val_loss: 8.0797e-04 - val_mae: 0.0287
Epoch 8/500
33/33 [==============================] - 1s 35ms/ste

23864.565417170525

In [13]:
tf.random.set_seed(42)

seq_length = 56
train_multask_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.to_numpy(),
    targets=mulvar_train[["bus", "rail"]][seq_length:],  # 2 targets per day
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_multask_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.to_numpy(),
    targets=mulvar_valid[["bus", "rail"]][seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

tf.random.set_seed(42)
multask_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    tf.keras.layers.Dense(2)
])

fit_and_evaluate(multask_model, train_multask_ds, valid_multask_ds,
                 learning_rate=0.02)

Epoch 1/500
33/33 [==============================] - 2s 24ms/step - loss: 0.0295 - mae: 0.1731 - val_loss: 0.0049 - val_mae: 0.0826
Epoch 2/500
33/33 [==============================] - 1s 21ms/step - loss: 0.0031 - mae: 0.0595 - val_loss: 0.0021 - val_mae: 0.0490
Epoch 3/500
33/33 [==============================] - 1s 29ms/step - loss: 0.0020 - mae: 0.0454 - val_loss: 0.0012 - val_mae: 0.0340
Epoch 4/500
33/33 [==============================] - 1s 26ms/step - loss: 0.0018 - mae: 0.0437 - val_loss: 0.0012 - val_mae: 0.0362
Epoch 5/500
33/33 [==============================] - 1s 17ms/step - loss: 0.0017 - mae: 0.0421 - val_loss: 0.0010 - val_mae: 0.0325
Epoch 6/500
33/33 [==============================] - 1s 16ms/step - loss: 0.0016 - mae: 0.0408 - val_loss: 0.0011 - val_mae: 0.0343
Epoch 7/500
33/33 [==============================] - 1s 16ms/step - loss: 0.0016 - mae: 0.0410 - val_loss: 8.7705e-04 - val_mae: 0.0286
Epoch 8/500
33/33 [==============================] - 1s 17ms/step - loss

24582.993239164352

In [14]:
Y_preds_valid = multask_model.predict(valid_multask_ds)
for idx, name in enumerate(["bus", "rail"]):
    mae = 1e6 * tf.keras.metrics.mean_absolute_error(
        mulvar_valid[name][seq_length:], Y_preds_valid[:, idx])
    print(name, int(mae))

3/3 [==============================] - 0s 9ms/step
bus 25358
rail 23807


# Data preprocessing with windowsing

In [15]:
def to_windows(dataset, length):
    dataset = dataset.window(length, shift=1, drop_remainder=True)
    return dataset.flat_map(lambda window_ds: window_ds.batch(length))

In [16]:
my_series = tf.data.Dataset.range(7)
for i in my_series:
  print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)


In [17]:
dataset = to_windows(to_windows(my_series, 3), 4)
dataset

<_FlatMapDataset element_spec=TensorSpec(shape=(None, None), dtype=tf.int64, name=None)>

In [18]:
list(dataset)

[<tf.Tensor: shape=(4, 3), dtype=int64, numpy=
 array([[0, 1, 2],
        [1, 2, 3],
        [2, 3, 4],
        [3, 4, 5]])>,
 <tf.Tensor: shape=(4, 3), dtype=int64, numpy=
 array([[1, 2, 3],
        [2, 3, 4],
        [3, 4, 5],
        [4, 5, 6]])>]

In [19]:
dataset = dataset.map(lambda S: (S[:, 0], S[:, 1:]))
list(dataset)

[(<tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 1, 2, 3])>,
  <tf.Tensor: shape=(4, 2), dtype=int64, numpy=
  array([[1, 2],
         [2, 3],
         [3, 4],
         [4, 5]])>),
 (<tf.Tensor: shape=(4,), dtype=int64, numpy=array([1, 2, 3, 4])>,
  <tf.Tensor: shape=(4, 2), dtype=int64, numpy=
  array([[2, 3],
         [3, 4],
         [4, 5],
         [5, 6]])>)]

In [20]:
def to_seq2seq_dataset(series, seq_length=56, ahead=14, target_col=1,
                       batch_size=32, shuffle=False, seed=None):
    ds = to_windows(tf.data.Dataset.from_tensor_slices(series), ahead + 1)
    ds = to_windows(ds, seq_length).map(lambda S: (S[:, 0], S[:, 1:, 1]))
    if shuffle:
        ds = ds.shuffle(8 * batch_size, seed=seed)
    return ds.batch(batch_size)

In [21]:
seq2seq_train = to_seq2seq_dataset(mulvar_train, shuffle=True, seed=42)
seq2seq_valid = to_seq2seq_dataset(mulvar_valid)

# Simple RNNs

In [22]:
tf.random.set_seed(42)
seq2seq_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[None, 5]),
    tf.keras.layers.Dense(14)
    # equivalent: tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(14))
    # also equivalent: tf.keras.layers.Conv1D(14, kernel_size=1)
])

In [23]:
fit_and_evaluate(seq2seq_model, seq2seq_train, seq2seq_valid,  learning_rate=0.1)

Epoch 1/500
33/33 [==============================] - 4s 70ms/step - loss: 0.0509 - mae: 0.2335 - val_loss: 0.0157 - val_mae: 0.1335
Epoch 2/500
33/33 [==============================] - 1s 22ms/step - loss: 0.0121 - mae: 0.1214 - val_loss: 0.0108 - val_mae: 0.1085
Epoch 3/500
33/33 [==============================] - 1s 23ms/step - loss: 0.0088 - mae: 0.1001 - val_loss: 0.0088 - val_mae: 0.0950
Epoch 4/500
33/33 [==============================] - 1s 23ms/step - loss: 0.0071 - mae: 0.0869 - val_loss: 0.0075 - val_mae: 0.0859
Epoch 5/500
33/33 [==============================] - 1s 23ms/step - loss: 0.0060 - mae: 0.0780 - val_loss: 0.0061 - val_mae: 0.0749
Epoch 6/500
33/33 [==============================] - 1s 22ms/step - loss: 0.0055 - mae: 0.0732 - val_loss: 0.0061 - val_mae: 0.0757
Epoch 7/500
33/33 [==============================] - 1s 21ms/step - loss: 0.0052 - mae: 0.0695 - val_loss: 0.0058 - val_mae: 0.0722
Epoch 8/500
33/33 [==============================] - 1s 24ms/step - loss: 0.

47073.99010658264

In [25]:
import numpy as np
X = mulvar_valid.to_numpy()[np.newaxis, :seq_length]
y_pred_14 = seq2seq_model.predict(X)[0, -1]  # only the last time step's output

1/1 [==============================] - 0s 325ms/step


In [26]:
Y_pred_valid = seq2seq_model.predict(seq2seq_valid)
for ahead in range(14):
    preds = pd.Series(Y_pred_valid[:-1, -1, ahead],
                      index=mulvar_valid.index[56 + ahead : -14 + ahead])
    mae = (preds - mulvar_valid["rail"]).abs().mean() * 1e6
    print(f"MAE for +{ahead + 1}: {mae:,.0f}")

3/3 [==============================] - 1s 63ms/step
MAE for +1: 24,370
MAE for +2: 29,969
MAE for +3: 30,494
MAE for +4: 30,544
MAE for +5: 31,998
MAE for +6: 34,553
MAE for +7: 34,312
MAE for +8: 33,917
MAE for +9: 34,001
MAE for +10: 33,706
MAE for +11: 38,459
MAE for +12: 37,584
MAE for +13: 36,796
MAE for +14: 34,641


# LSTMs

In [27]:
tf.random.set_seed(42)  # extra code – ensures reproducibility
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 5]),
    tf.keras.layers.Dense(14)
])

In [28]:
fit_and_evaluate(lstm_model, seq2seq_train, seq2seq_valid,
                 learning_rate=0.1, epochs=50)

Epoch 1/50
33/33 [==============================] - 4s 54ms/step - loss: 0.0575 - mae: 0.2566 - val_loss: 0.0176 - val_mae: 0.1584
Epoch 2/50
33/33 [==============================] - 1s 36ms/step - loss: 0.0165 - mae: 0.1546 - val_loss: 0.0170 - val_mae: 0.1437
Epoch 3/50
33/33 [==============================] - 2s 53ms/step - loss: 0.0152 - mae: 0.1486 - val_loss: 0.0162 - val_mae: 0.1404
Epoch 4/50
33/33 [==============================] - 2s 55ms/step - loss: 0.0146 - mae: 0.1442 - val_loss: 0.0155 - val_mae: 0.1369
Epoch 5/50
33/33 [==============================] - 2s 66ms/step - loss: 0.0140 - mae: 0.1407 - val_loss: 0.0149 - val_mae: 0.1337
Epoch 6/50
33/33 [==============================] - 3s 75ms/step - loss: 0.0134 - mae: 0.1373 - val_loss: 0.0144 - val_mae: 0.1308
Epoch 7/50
33/33 [==============================] - 2s 59ms/step - loss: 0.0129 - mae: 0.1340 - val_loss: 0.0138 - val_mae: 0.1289
Epoch 8/50
33/33 [==============================] - 2s 47ms/step - loss: 0.0125 - m

90553.1793832779

# GRUs

In [29]:
tf.random.set_seed(42)  # extra code – ensures reproducibility
gru_model = tf.keras.Sequential([
    tf.keras.layers.GRU(32, return_sequences=True, input_shape=[None, 5]),
    tf.keras.layers.Dense(14)
])

In [30]:
fit_and_evaluate(gru_model, seq2seq_train, seq2seq_valid,
                 learning_rate=0.1, epochs=50)

Epoch 1/50
33/33 [==============================] - 5s 58ms/step - loss: 0.0553 - mae: 0.2589 - val_loss: 0.0192 - val_mae: 0.1634
Epoch 2/50
33/33 [==============================] - 1s 38ms/step - loss: 0.0159 - mae: 0.1461 - val_loss: 0.0154 - val_mae: 0.1385
Epoch 3/50
33/33 [==============================] - 1s 38ms/step - loss: 0.0133 - mae: 0.1348 - val_loss: 0.0140 - val_mae: 0.1281
Epoch 4/50
33/33 [==============================] - 1s 38ms/step - loss: 0.0121 - mae: 0.1267 - val_loss: 0.0130 - val_mae: 0.1229
Epoch 5/50
33/33 [==============================] - 1s 38ms/step - loss: 0.0113 - mae: 0.1213 - val_loss: 0.0122 - val_mae: 0.1190
Epoch 6/50
33/33 [==============================] - 1s 40ms/step - loss: 0.0107 - mae: 0.1169 - val_loss: 0.0115 - val_mae: 0.1166
Epoch 7/50
33/33 [==============================] - 2s 62ms/step - loss: 0.0103 - mae: 0.1141 - val_loss: 0.0112 - val_mae: 0.1140
Epoch 8/50
33/33 [==============================] - 2s 39ms/step - loss: 0.0100 - m

79424.4185090065

## Resources:
- [Hands-on Machine Learning with Scikit-Learn & TensorFlow](https://github.com/ageron/handson-ml3/blob/main/15_processing_sequences_using_rnns_and_cnns.ipynb)


Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Instagram](https://www.instagram.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎